<a href="https://colab.research.google.com/github/stevenfox/products_classification/blob/main/BrandMind_Stefanos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Classification on shopping products using CNN with TF and Keras

  Classify a given product based on 29 categories.

In [ ]:
# TensorFlow and tf.keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


import os
import matplotlib.image as mpimg
import cv2
from imutils import paths
# import pickle


2.4.1


Paths


In [ ]:
dataset_dir= '/content/drive/MyDrive/Colab Notebooks/BrandMind_Assessment/Products'

In [ ]:
data = []
labels = []

In [ ]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(dataset_dir)))

In [ ]:
#total number images
total_number_of_images = len(imagePaths)
print("Total number of images----->",total_number_of_images)
random.shuffle(imagePaths)

Total number of images-----> 9398


In [ ]:
# loop over the shuffled input images
for imagePath in imagePaths:

	#Read the image into a numpy array using opencv
	#all the read images are of different shapes
	image = cv2.imread(imagePath)

	#resize the image to be 32x32 pixels (ignoring aspect ratio)
	#After reshape size of all the images will become 32x32x3
	#Total number of pixels in every image = 32x32x3=3072
	image = cv2.resize(image, (32, 32))

	#flatten converts every 3D image (32x32x3) into 1D numpy array of shape (3072,)
	#(3072,) is the shape of the flatten image
	#(3072,) shape means 3072 columns and 1 row
	image_flatten = image.flatten()

	#Append each image data 1D array to the data list
	data.append(image_flatten)

	# extract the class label from the image path and update the
	label = imagePath.split(os.path.sep)[-2]

	#Append each image label to the labels list
	labels.append(label)
# scale the raw pixel intensities to the range [0, 1]
#convert the data and label list to numpy array
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
# partition the data into training, testing and validation splits using 60% of
# the data for training, 20% for testing and the remaining 20% for validating
# train_test_split is a scikit-learn's function which helps us to split images into three devisions of training, testing and validation respectively

(X_train, X_test, y_train, y_test) = train_test_split(data, labels, test_size=0.2, random_state=1) # initial split of 80% / 20%

(X_train, X_val, y_train, y_val )= train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


print("Amount of training images --->",len(X_train),",","Amount of training labels --->",len(y_train))
print("Amount of testing images --->",len(X_test),",","Amount of testing labels --->",len(y_test))
print("Amount of validating images --->",len(X_val),",","Amount of testing labels --->",len(y_val))


Amount of training images ---> 5638 , Amount of training labels ---> 5638
Amount of testing images ---> 1880 , Amount of testing labels ---> 1880
Amount of validating images ---> 1880 , Amount of testing labels ---> 1880


In [ ]:
# convert the labels from integers to vectors 
# perform One hot encoding of all the labels using scikit-learn's function LabelBinarizer
# LabelBinarizer fit_transform finds all the labels

lb = preprocessing.LabelBinarizer()
trainY = lb.fit_transform(y_train)
testY = lb.transform(y_test)
valY = lb.transform(y_val)


In [ ]:
print("\n")
print ("Classes found to train",)
train_classes = lb.classes_
print(train_classes)
binary_rep_each_class = lb.transform(train_classes)
print("Binary representation of each class")
print(binary_rep_each_class)
print("\n")



Classes found to train
['beanie' 'bikini' 'blazers' 'boat-shoes' 'boatneck-tops' 'bodysuit' 'boots' 'bowtie' 'bracelets' 'business-suits'
 'cami-tops' 'chelsea-boots' 'coats' 'crop-tops' 'demin-jackets' 'earrings' 'evening-gowns' 'gloves' 'hoodies' 'pumps'
 'scarf' 'skirts' 'sneakers' 'stilettos' 'sunglasses' 'trousers' 'tuxedos' 'waistcoat' 'wristwatch']
Binary representation of each class
[[1 0 0 0 ... 0 0 0 0]
 [0 1 0 0 ... 0 0 0 0]
 [0 0 1 0 ... 0 0 0 0]
 [0 0 0 1 ... 0 0 0 0]
 ...
 [0 0 0 0 ... 1 0 0 0]
 [0 0 0 0 ... 0 1 0 0]
 [0 0 0 0 ... 0 0 1 0]
 [0 0 0 0 ... 0 0 0 1]]




## Deep learning libraries

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras import backend as K


In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

INIT_LR = 0.01
EPOCHS = 75
opt = SGD(lr=INIT_LR)

model = Sequential()
model.add(Flatten(input_shape=trainX.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_classes), activation='sigmoid'))
model.summary()

model.compile(loss="categorical_crossentropy", 
              optimizer=opt,metrics=["accuracy"])


model.fit(X_train, trainY,
          epochs=50,
          batch_size=batch_size,
          validation_data=(X_val, valY))
model.save_weights('bottleneck_fc_model.h5')

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               786688    
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 29)                7453      
Total params: 794,141
Trainable params: 794,141
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
353/353 [==============================] - 3s 7ms/step - loss: 3.2902 - accuracy: 0.0955 - val_loss: 2.5907 - val_accuracy: 0.2622
Epoch 2/50
353/353 [==============================] - 2s 6ms/step - loss: 2.6837 - accuracy: 0.2333 - v